In [2]:
#scrape data from wiki page
from bs4 import BeautifulSoup
import requests
import pandas as pd

df = pd.DataFrame(columns=['Postcode', 'Borough', 'Neighbourhood'])

with open('wikipage.htm') as html_file:
    soup = BeautifulSoup(html_file,'lxml')

i=0    
for code in soup.find('div',class_='mw-parser-output').table.tbody.find_all('tr'):
    list0=list()
    for element in code.find_all('td'):
        list0.append(element.text.strip())
    if list0==list():continue
    df.loc[i]=list0
    i=i+1

pd.options.mode.chained_assignment = None

#get rid of the ones with no assigned borough
df_bo=df[df['Borough']!='Not assigned']


#deal with no neighbourhood
index=(df_bo['Neighbourhood']=='Not assigned')
index
df_bo.loc[index,'Neighbourhood']=df_bo.loc[index,'Borough']




#combine the ones with same postcode
df_nei=df_bo.groupby(['Postcode','Borough']).Neighbourhood.apply(lambda x : ",".join(x)).to_frame()

#final data
df_canada=df_nei.reset_index()

data = pd.read_csv("Geospatial_Coordinates.csv") 
data.rename(columns={'Postal Code':'Postcode'}, inplace=True)

df_merge =pd.merge(df_canada,data,how='left',on='Postcode')
df_merge.head()


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [3]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 


!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library




Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



In [4]:
address = 'Toronto,Canada'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [5]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_merge['Latitude'], df_merge['Longitude'], df_merge['Borough'], df_merge['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto) 
    
map_Toronto